In [1]:
!pip install torchsummary
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchsummary import summary
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torchvision.datasets import ImageFolder
from tqdm import notebook
from torch.utils.data import DataLoader, random_split
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
import time
import os
import copy

# Data

In [2]:
data_dir='G:\DCDA-Net\Data'
print(os.listdir(data_dir))

['0', '1']


In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [4]:
transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Resize(size=(224,224)),
                transforms.Normalize((0.3433, 0.7419, 0.5770), (0.3453, 0.0661, 0.2652))
                ])

In [5]:
dataset=ImageFolder(data_dir,transform=transform)
torch.manual_seed(42)
train_ds, val_ds, test_ds = random_split(dataset, [0.8,0.15,0.05])
len(train_ds),len(val_ds),len(test_ds)

(4867, 912, 304)

In [6]:
trainloader =DataLoader(train_ds, batch_size=16, shuffle=True,num_workers=4)
valloader = DataLoader(val_ds, batch_size=32, shuffle=True,num_workers=4)

In [11]:
def evaluation(dataloader, model):
    total, correct = 0, 0
    model.eval()
    for data in dataloader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, pred = torch.max(outputs.data, 1)

        total += labels.size(0)
        correct += (pred == labels).sum().item()
    return 100 * correct / total

In [10]:
def train_func(model, loss_fn, opt, trainloader, valloader, scheduler , epochs):
    loss_epoch_arr = []
    train_accuracy_lis = []
    val_accuracy_lis = []
    val_loss_epoch_arr = []

    max_epochs = epochs
    min_loss = 1000
    model.train()

    for epoch in notebook.tqdm(range(max_epochs), total=max_epochs, unit="epochs"):
        model.train()
        for i, data in enumerate(trainloader, 0):
    
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            opt.zero_grad()
            outputs = model(inputs)
            loss = loss_fn(outputs, labels)
            loss.backward()
            opt.step()
            scheduler.step()

            if min_loss > loss.item():
                min_loss = loss.item()
                best_model = copy.deepcopy(model.state_dict())
                print("Min loss %0.2f" %min_loss)
            del inputs, labels, outputs
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
            if i % 2000 == 0:
                print('Iteration: %d, Loss: %0.2f' % (i, loss.item()))
            
       
        loss_epoch_arr.append(loss.item())
        train_accuracy = evaluation(trainloader, model)
        val_accuracy = evaluation(valloader, model)
        print('Epoch: %d/%d, Test acc: %0.2f, Train acc: %0.2f' % (
              epoch, max_epochs, val_accuracy, train_accuracy))
        train_accuracy_lis.append(train_accuracy)
        val_accuracy_lis.append(val_accuracy)
    
        model.eval()
        with torch.no_grad():
            for i, data in enumerate(valloader, 0):
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = loss_fn(outputs, labels)

        del inputs, labels, outputs
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

        val_loss_epoch_arr.append(loss.item())
    return best_model, loss_epoch_arr, train_accuracy_lis, val_accuracy_lis, val_loss_epoch_arr

# Resnet50

In [9]:
resnet = models.resnet50(weights=True)

C:\Users\BTBM\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [10]:
for param in resnet.parameters():
    param.requires_grad=False

In [11]:
num_features = resnet.fc.in_features
new_classifier = nn.Sequential(
    nn.Linear(num_features, 256), nn.ReLU(), nn.Dropout(0.2),
    nn.Linear(256, 2))
resnet.fc = new_classifier

In [12]:
resnet =resnet.to(device)
loss_fn = nn.CrossEntropyLoss()
opt = optim.Adam(resnet.parameters(), lr=0.001)
summary(resnet, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

In [13]:
gamma = 0.9  # The decay factor for each epoch
scheduler = lr_scheduler.ExponentialLR(opt, gamma=gamma)

In [14]:
best_model, loss_epoch_arr, train_accuracy_lis, val_accuracy_lis, val_loss_epoch_arr = train_func(resnet, loss_fn, opt, trainloader, valloader, scheduler , epochs=50
                                                                            )      

  0%|          | 0/50 [00:00<?, ?epochs/s]

Min loss 0.68
Iteration: 0, Loss: 0.68
Min loss 0.64
Min loss 0.61
Min loss 0.59
Epoch: 0/50, Test acc: 49.12, Train acc: 52.89
Iteration: 0, Loss: 0.70
Min loss 0.57
Epoch: 1/50, Test acc: 48.25, Train acc: 52.97
Iteration: 0, Loss: 0.62
Epoch: 2/50, Test acc: 49.56, Train acc: 53.03
Iteration: 0, Loss: 0.78
Epoch: 3/50, Test acc: 48.90, Train acc: 53.46
Iteration: 0, Loss: 0.70
Epoch: 4/50, Test acc: 50.66, Train acc: 53.19
Iteration: 0, Loss: 0.70
Min loss 0.55
Epoch: 5/50, Test acc: 49.56, Train acc: 53.36
Iteration: 0, Loss: 0.71
Epoch: 6/50, Test acc: 49.67, Train acc: 52.76
Iteration: 0, Loss: 0.70
Epoch: 7/50, Test acc: 50.33, Train acc: 52.66
Iteration: 0, Loss: 0.72
Epoch: 8/50, Test acc: 49.23, Train acc: 53.40
Iteration: 0, Loss: 0.67
Epoch: 9/50, Test acc: 49.12, Train acc: 53.26
Iteration: 0, Loss: 0.64
Epoch: 10/50, Test acc: 49.56, Train acc: 53.24
Iteration: 0, Loss: 0.72
Epoch: 11/50, Test acc: 49.89, Train acc: 52.64
Iteration: 0, Loss: 0.69
Epoch: 12/50, Test acc: 5

In [15]:
testloader = DataLoader(test_ds, batch_size=32, shuffle=False)

In [16]:
evaluation(testloader,resnet)

55.5921052631579

# Resnet-152

In [17]:
resnet = models.resnet152(weights=True)

C:\Users\BTBM\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [18]:
for param in resnet.parameters():
    param.requires_grad=False

In [19]:
num_features = resnet.fc.in_features
new_classifier = nn.Sequential(
    nn.Linear(num_features, 256), nn.ReLU(), nn.Dropout(0.2),
    nn.Linear(256, 2))
resnet.fc = new_classifier

In [20]:
resnet =resnet.to(device)
loss_fn = nn.CrossEntropyLoss()
opt = optim.Adam(resnet.parameters(), lr=0.001)
summary(resnet, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

In [21]:
gamma = 0.9  # The decay factor for each epoch
scheduler = lr_scheduler.ExponentialLR(opt, gamma=gamma)


In [22]:
best_model, loss_epoch_arr, train_accuracy_lis, val_accuracy_lis, val_loss_epoch_arr = train_func(resnet, loss_fn, opt, trainloader, valloader, scheduler , epochs=50
                                                                            )      

  0%|          | 0/50 [00:00<?, ?epochs/s]

Min loss 0.69
Iteration: 0, Loss: 0.69
Min loss 0.66
Min loss 0.65
Min loss 0.62
Min loss 0.60
Min loss 0.57
Min loss 0.55
Epoch: 0/50, Test acc: 59.32, Train acc: 58.35
Iteration: 0, Loss: 0.70
Epoch: 1/50, Test acc: 58.88, Train acc: 58.39
Iteration: 0, Loss: 0.71
Epoch: 2/50, Test acc: 59.32, Train acc: 58.80
Iteration: 0, Loss: 0.67
Epoch: 3/50, Test acc: 59.54, Train acc: 57.61
Iteration: 0, Loss: 0.59
Epoch: 4/50, Test acc: 58.55, Train acc: 58.68
Iteration: 0, Loss: 0.82
Epoch: 5/50, Test acc: 59.10, Train acc: 58.66
Iteration: 0, Loss: 0.64
Epoch: 6/50, Test acc: 58.99, Train acc: 58.02
Iteration: 0, Loss: 0.72
Epoch: 7/50, Test acc: 58.77, Train acc: 58.17
Iteration: 0, Loss: 0.64
Epoch: 8/50, Test acc: 59.65, Train acc: 58.15
Iteration: 0, Loss: 0.66
Epoch: 9/50, Test acc: 59.87, Train acc: 58.17
Iteration: 0, Loss: 0.63
Epoch: 10/50, Test acc: 58.66, Train acc: 58.35
Iteration: 0, Loss: 0.69
Epoch: 11/50, Test acc: 59.32, Train acc: 57.94
Iteration: 0, Loss: 0.75
Min loss 0.

In [23]:
testloader = DataLoader(test_ds, batch_size=32, shuffle=False)

In [24]:
evaluation(testloader,resnet)

57.23684210526316

# Resnet 18

In [25]:
resnet = models.resnet18(weights=True)

C:\Users\BTBM\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [26]:
for param in resnet.parameters():
    param.requires_grad=False

In [27]:
num_features = resnet.fc.in_features
new_classifier = nn.Sequential(
    nn.Linear(num_features, 256), nn.ReLU(), nn.Dropout(0.2),
    nn.Linear(256, 2))
resnet.fc = new_classifier

In [28]:
resnet =resnet.to(device)
loss_fn = nn.CrossEntropyLoss()
opt = optim.Adam(resnet.parameters(), lr=0.001)
summary(resnet, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
       BasicBlock-11           [-1, 64, 56, 56]               0
           Conv2d-12           [-1, 64, 56, 56]          36,864
      BatchNorm2d-13           [-1, 64, 56, 56]             128
             ReLU-14           [-1, 64,

In [29]:
gamma = 0.9  # The decay factor for each epoch
scheduler = lr_scheduler.ExponentialLR(opt, gamma=gamma)


In [30]:
best_model, loss_epoch_arr, train_accuracy_lis, val_accuracy_lis, val_loss_epoch_arr = train_func(resnet, loss_fn, opt, trainloader, valloader, scheduler , epochs=50
                                                                            )      

  0%|          | 0/50 [00:00<?, ?epochs/s]

Min loss 0.73
Iteration: 0, Loss: 0.73
Min loss 0.71
Min loss 0.64
Min loss 0.62
Min loss 0.61
Min loss 0.58
Min loss 0.58
Epoch: 0/50, Test acc: 59.10, Train acc: 59.32
Iteration: 0, Loss: 0.66
Min loss 0.56
Min loss 0.56
Epoch: 1/50, Test acc: 57.46, Train acc: 57.76
Iteration: 0, Loss: 0.68
Min loss 0.54
Epoch: 2/50, Test acc: 58.88, Train acc: 59.50
Iteration: 0, Loss: 0.72
Min loss 0.54
Epoch: 3/50, Test acc: 59.10, Train acc: 58.93
Iteration: 0, Loss: 0.67
Epoch: 4/50, Test acc: 59.98, Train acc: 59.46
Iteration: 0, Loss: 0.73
Epoch: 5/50, Test acc: 59.10, Train acc: 59.15
Iteration: 0, Loss: 0.70
Min loss 0.53
Epoch: 6/50, Test acc: 59.10, Train acc: 59.24
Iteration: 0, Loss: 0.67
Epoch: 7/50, Test acc: 60.20, Train acc: 59.38
Iteration: 0, Loss: 0.68
Epoch: 8/50, Test acc: 58.88, Train acc: 59.28
Iteration: 0, Loss: 0.71
Epoch: 9/50, Test acc: 58.44, Train acc: 58.56
Iteration: 0, Loss: 0.67
Epoch: 10/50, Test acc: 57.79, Train acc: 58.70
Iteration: 0, Loss: 0.64
Epoch: 11/50, 

In [31]:
testloader = DataLoader(test_ds, batch_size=32, shuffle=False)

In [32]:
evaluation(testloader,resnet)

63.48684210526316

# resnet-34

In [33]:
resnet = models.resnet34(weights=True)

C:\Users\BTBM\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [34]:
for param in resnet.parameters():
    param.requires_grad=False

In [35]:
num_features = resnet.fc.in_features
new_classifier = nn.Sequential(
    nn.Linear(num_features, 256), nn.ReLU(), nn.Dropout(0.2),
    nn.Linear(256, 2))
resnet.fc = new_classifier

In [36]:
resnet =resnet.to(device)
loss_fn = nn.CrossEntropyLoss()
opt = optim.Adam(resnet.parameters(), lr=0.001)
summary(resnet, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
       BasicBlock-11           [-1, 64, 56, 56]               0
           Conv2d-12           [-1, 64, 56, 56]          36,864
      BatchNorm2d-13           [-1, 64, 56, 56]             128
             ReLU-14           [-1, 64,

In [37]:
gamma = 0.9  # The decay factor for each epoch
scheduler = lr_scheduler.ExponentialLR(opt, gamma=gamma)


In [38]:
best_model, loss_epoch_arr, train_accuracy_lis, val_accuracy_lis, val_loss_epoch_arr = train_func(resnet, loss_fn, opt, trainloader, valloader, scheduler , epochs=50
                                                                            )      

  0%|          | 0/50 [00:00<?, ?epochs/s]

Min loss 0.65
Iteration: 0, Loss: 0.65
Min loss 0.59
Min loss 0.47
Epoch: 0/50, Test acc: 54.06, Train acc: 53.77
Iteration: 0, Loss: 0.67
Epoch: 1/50, Test acc: 52.85, Train acc: 52.93
Iteration: 0, Loss: 0.74
Epoch: 2/50, Test acc: 52.96, Train acc: 53.69
Iteration: 0, Loss: 0.70
Epoch: 3/50, Test acc: 53.29, Train acc: 53.54
Iteration: 0, Loss: 0.72
Epoch: 4/50, Test acc: 53.73, Train acc: 53.54
Iteration: 0, Loss: 0.74
Epoch: 5/50, Test acc: 53.07, Train acc: 53.52
Iteration: 0, Loss: 0.70
Epoch: 6/50, Test acc: 53.29, Train acc: 53.59
Iteration: 0, Loss: 0.70
Epoch: 7/50, Test acc: 53.40, Train acc: 53.87
Iteration: 0, Loss: 0.71
Epoch: 8/50, Test acc: 54.39, Train acc: 53.81
Iteration: 0, Loss: 0.76
Epoch: 9/50, Test acc: 53.62, Train acc: 53.54
Iteration: 0, Loss: 0.69
Epoch: 10/50, Test acc: 53.73, Train acc: 53.15
Iteration: 0, Loss: 0.68
Epoch: 11/50, Test acc: 53.40, Train acc: 53.11
Iteration: 0, Loss: 0.66
Epoch: 12/50, Test acc: 53.40, Train acc: 53.36
Iteration: 0, Loss:

In [39]:
testloader = DataLoader(test_ds, batch_size=32, shuffle=False)

In [40]:
evaluation(testloader,resnet)

53.28947368421053

# resnet 101

In [41]:
resnet = models.resnet101(weights=True)

C:\Users\BTBM\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [42]:
for param in resnet.parameters():
    param.requires_grad=False

In [43]:
num_features = resnet.fc.in_features
new_classifier = nn.Sequential(
    nn.Linear(num_features, 256), nn.ReLU(), nn.Dropout(0.2),
    nn.Linear(256, 2))
resnet.fc = new_classifier

In [44]:
resnet =resnet.to(device)
loss_fn = nn.CrossEntropyLoss()
opt = optim.Adam(resnet.parameters(), lr=0.001)
summary(resnet, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

In [45]:
gamma = 0.9  # The decay factor for each epoch
scheduler = lr_scheduler.ExponentialLR(opt, gamma=gamma)


In [46]:
best_model, loss_epoch_arr, train_accuracy_lis, val_accuracy_lis, val_loss_epoch_arr = train_func(resnet, loss_fn, opt, trainloader, valloader, scheduler , epochs=50
                                                                            )      

  0%|          | 0/50 [00:00<?, ?epochs/s]

Min loss 0.74
Iteration: 0, Loss: 0.74
Min loss 0.73
Min loss 0.72
Min loss 0.69
Min loss 0.69
Min loss 0.65
Min loss 0.65
Min loss 0.58
Min loss 0.58
Epoch: 0/50, Test acc: 55.26, Train acc: 56.40
Iteration: 0, Loss: 0.76
Epoch: 1/50, Test acc: 54.50, Train acc: 55.78
Iteration: 0, Loss: 0.64
Min loss 0.57
Min loss 0.56
Min loss 0.56
Epoch: 2/50, Test acc: 55.04, Train acc: 56.69
Iteration: 0, Loss: 0.68
Min loss 0.55
Epoch: 3/50, Test acc: 54.82, Train acc: 55.56
Iteration: 0, Loss: 0.81
Epoch: 4/50, Test acc: 55.48, Train acc: 56.46
Iteration: 0, Loss: 0.74
Epoch: 5/50, Test acc: 55.37, Train acc: 56.32
Iteration: 0, Loss: 0.62
Min loss 0.54
Epoch: 6/50, Test acc: 54.71, Train acc: 55.68
Iteration: 0, Loss: 0.67
Epoch: 7/50, Test acc: 54.50, Train acc: 55.66
Iteration: 0, Loss: 0.69
Min loss 0.50
Epoch: 8/50, Test acc: 55.70, Train acc: 56.34
Iteration: 0, Loss: 0.69
Epoch: 9/50, Test acc: 54.28, Train acc: 55.64
Iteration: 0, Loss: 0.76
Epoch: 10/50, Test acc: 56.14, Train acc: 56.

In [47]:
testloader = DataLoader(test_ds, batch_size=32, shuffle=False)

In [48]:
evaluation(testloader,resnet)

51.31578947368421

# VGG-16

In [49]:
vgg16 = models.vgg16_bn(pretrained=True)

C:\Users\BTBM\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\BTBM\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_BN_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_BN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [50]:
for param in vgg16.features.parameters():
    param.requires_grad = False

In [51]:
num_features = vgg16.classifier[6].in_features
vgg16.classifier[6] = nn.Linear(num_features, 2)

In [52]:
vgg16 = vgg16.to(device)
loss_fn = nn.CrossEntropyLoss()
opt = optim.Adam(vgg16.parameters(), lr=0.001)

In [53]:
gamma = 0.9  # The decay factor for each epoch
scheduler = lr_scheduler.ExponentialLR(opt, gamma=gamma)

In [54]:
best_model, loss_epoch_arr, train_accuracy_lis, val_accuracy_lis, val_loss_epoch_arr = train_func(vgg16, loss_fn, opt, trainloader, valloader, scheduler , epochs=50
                                                                            )      

  0%|          | 0/50 [00:00<?, ?epochs/s]

Min loss 0.77
Iteration: 0, Loss: 0.77
Min loss 0.72
Min loss 0.69
Min loss 0.67
Min loss 0.51
Min loss 0.50
Min loss 0.48
Min loss 0.45
Min loss 0.40
Epoch: 0/50, Test acc: 61.84, Train acc: 62.44
Iteration: 0, Loss: 0.92
Epoch: 1/50, Test acc: 62.94, Train acc: 62.75
Iteration: 0, Loss: 0.65
Min loss 0.38
Epoch: 2/50, Test acc: 62.83, Train acc: 62.42
Iteration: 0, Loss: 0.83
Epoch: 3/50, Test acc: 63.05, Train acc: 62.83
Iteration: 0, Loss: 0.61
Epoch: 4/50, Test acc: 63.05, Train acc: 62.54
Iteration: 0, Loss: 0.86
Min loss 0.37
Epoch: 5/50, Test acc: 63.60, Train acc: 62.73
Iteration: 0, Loss: 0.76
Epoch: 6/50, Test acc: 63.60, Train acc: 62.91
Iteration: 0, Loss: 0.78
Epoch: 7/50, Test acc: 63.16, Train acc: 62.75
Iteration: 0, Loss: 0.90
Epoch: 8/50, Test acc: 63.38, Train acc: 62.54
Iteration: 0, Loss: 0.77
Epoch: 9/50, Test acc: 62.28, Train acc: 62.61
Iteration: 0, Loss: 1.05
Epoch: 10/50, Test acc: 63.82, Train acc: 62.75
Iteration: 0, Loss: 0.89
Epoch: 11/50, Test acc: 61.8

In [55]:
testloader = DataLoader(test_ds, batch_size=32, shuffle=False)

In [56]:
evaluation(testloader,vgg16)

59.86842105263158

# VGG-19

In [57]:
vgg19 = models.vgg19_bn(pretrained=True)

C:\Users\BTBM\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_BN_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_BN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [58]:
for param in vgg19.features.parameters():
    param.requires_grad = False

In [59]:
num_features = vgg19.classifier[6].in_features
vgg19.classifier[6] = nn.Linear(num_features, 2)

In [60]:
vgg19 = vgg19.to(device)
loss_fn = nn.CrossEntropyLoss()
opt = optim.Adam(vgg19.parameters(), lr=0.001)

In [61]:
gamma = 0.9  # The decay factor for each epoch
scheduler = lr_scheduler.ExponentialLR(opt, gamma=gamma)

In [62]:
best_model, loss_epoch_arr, train_accuracy_lis, val_accuracy_lis, val_loss_epoch_arr = train_func(vgg19, loss_fn, opt, trainloader, valloader, scheduler , epochs=50
                                                                            )      

  0%|          | 0/50 [00:00<?, ?epochs/s]

Min loss 0.73
Iteration: 0, Loss: 0.73
Min loss 0.65
Min loss 0.60
Min loss 0.54
Min loss 0.50
Epoch: 0/50, Test acc: 60.75, Train acc: 61.62
Iteration: 0, Loss: 0.75
Min loss 0.50
Epoch: 1/50, Test acc: 60.42, Train acc: 61.45
Iteration: 0, Loss: 0.83
Min loss 0.49
Epoch: 2/50, Test acc: 60.31, Train acc: 61.27
Iteration: 0, Loss: 0.68
Min loss 0.46
Min loss 0.44
Epoch: 3/50, Test acc: 60.31, Train acc: 61.11
Iteration: 0, Loss: 0.70
Min loss 0.41
Epoch: 4/50, Test acc: 60.42, Train acc: 61.48
Iteration: 0, Loss: 0.76
Epoch: 5/50, Test acc: 60.53, Train acc: 61.54
Iteration: 0, Loss: 0.65
Epoch: 6/50, Test acc: 60.86, Train acc: 61.68
Iteration: 0, Loss: 0.72
Epoch: 7/50, Test acc: 60.31, Train acc: 61.48
Iteration: 0, Loss: 0.65
Min loss 0.40
Epoch: 8/50, Test acc: 60.09, Train acc: 60.94
Iteration: 0, Loss: 0.81
Epoch: 9/50, Test acc: 60.96, Train acc: 61.48
Iteration: 0, Loss: 0.75
Epoch: 10/50, Test acc: 60.42, Train acc: 60.94
Iteration: 0, Loss: 0.83
Epoch: 11/50, Test acc: 60.3

In [63]:
testloader = DataLoader(test_ds, batch_size=32, shuffle=False)

In [64]:
evaluation(testloader,vgg19)

57.23684210526316

# Densenet121

In [65]:
densenet121 = models.densenet121(weights=True)

C:\Users\BTBM\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [66]:
for param in densenet121.features.parameters():
    param.requires_grad = False


In [67]:
num_features = densenet121.classifier.in_features
densenet121.classifier = nn.Linear(num_features, 2)

In [68]:
densenet121 = densenet121.to(device)
loss_fn = nn.CrossEntropyLoss()
opt = optim.Adam(densenet121.parameters(), lr=0.001)

In [69]:
gamma = 0.9  # The decay factor for each epoch
scheduler = lr_scheduler.ExponentialLR(opt, gamma=gamma)

In [70]:
best_model, loss_epoch_arr, train_accuracy_lis, val_accuracy_lis, val_loss_epoch_arr = train_func(densenet121, loss_fn, opt, trainloader, valloader, scheduler , epochs=50
                                                                            )      

  0%|          | 0/50 [00:00<?, ?epochs/s]

Min loss 0.68
Iteration: 0, Loss: 0.68
Min loss 0.68
Min loss 0.66
Min loss 0.59
Epoch: 0/50, Test acc: 46.82, Train acc: 48.67
Min loss 0.58
Iteration: 0, Loss: 0.58
Epoch: 1/50, Test acc: 46.71, Train acc: 48.59
Iteration: 0, Loss: 0.67
Epoch: 2/50, Test acc: 47.15, Train acc: 49.31
Iteration: 0, Loss: 0.78
Epoch: 3/50, Test acc: 47.04, Train acc: 48.92
Iteration: 0, Loss: 0.70
Epoch: 4/50, Test acc: 46.93, Train acc: 48.45
Iteration: 0, Loss: 0.77
Min loss 0.58
Epoch: 5/50, Test acc: 45.94, Train acc: 48.59
Iteration: 0, Loss: 0.61
Epoch: 6/50, Test acc: 47.59, Train acc: 48.96
Iteration: 0, Loss: 0.69
Min loss 0.58
Epoch: 7/50, Test acc: 46.38, Train acc: 48.70
Iteration: 0, Loss: 0.70
Min loss 0.58
Epoch: 8/50, Test acc: 47.48, Train acc: 49.04
Iteration: 0, Loss: 0.69
Epoch: 9/50, Test acc: 46.93, Train acc: 48.84
Iteration: 0, Loss: 0.77
Epoch: 10/50, Test acc: 46.60, Train acc: 49.11
Iteration: 0, Loss: 0.75
Min loss 0.58
Epoch: 11/50, Test acc: 47.26, Train acc: 49.17
Iteratio

In [71]:
testloader = DataLoader(test_ds, batch_size=32, shuffle=False)

In [72]:
evaluation(testloader,densenet121)

51.31578947368421

# DenseNet201

In [9]:
densenet201 = models.densenet201(pretrained=True)

C:\Users\BTBM\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\BTBM\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet201_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet201_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [74]:
for param in densenet201.features.parameters():
    param.requires_grad = False

In [75]:
num_features = densenet201.classifier.in_features
densenet201.classifier = nn.Linear(num_features, 2)

In [76]:
densenet201 = densenet201.to(device)
loss_fn = nn.CrossEntropyLoss()
opt = optim.Adam(densenet201.parameters(), lr=0.001)

In [77]:
gamma = 0.9  # The decay factor for each epoch
scheduler = lr_scheduler.ExponentialLR(opt, gamma=gamma)

In [78]:
best_model, loss_epoch_arr, train_accuracy_lis, val_accuracy_lis, val_loss_epoch_arr = train_func(densenet201, loss_fn, opt, trainloader, valloader, scheduler , epochs=100
                                                                            )      

  0%|          | 0/50 [00:00<?, ?epochs/s]

Min loss 0.69
Iteration: 0, Loss: 0.69
Min loss 0.64
Min loss 0.58
Min loss 0.58
Min loss 0.55
Min loss 0.55
Min loss 0.54
Min loss 0.48
Epoch: 0/50, Test acc: 65.13, Train acc: 67.50
Iteration: 0, Loss: 0.65
Epoch: 1/50, Test acc: 64.91, Train acc: 67.13
Iteration: 0, Loss: 0.72
Epoch: 2/50, Test acc: 65.24, Train acc: 67.56
Iteration: 0, Loss: 0.63
Min loss 0.47
Min loss 0.46
Epoch: 3/50, Test acc: 65.46, Train acc: 67.43
Iteration: 0, Loss: 0.65
Epoch: 4/50, Test acc: 64.91, Train acc: 67.50
Iteration: 0, Loss: 0.60
Epoch: 5/50, Test acc: 65.79, Train acc: 67.68
Iteration: 0, Loss: 0.59
Epoch: 6/50, Test acc: 65.46, Train acc: 67.87
Iteration: 0, Loss: 0.56
Epoch: 7/50, Test acc: 65.02, Train acc: 67.17
Iteration: 0, Loss: 0.58
Epoch: 8/50, Test acc: 66.01, Train acc: 67.84
Iteration: 0, Loss: 0.59
Epoch: 9/50, Test acc: 64.80, Train acc: 67.41
Iteration: 0, Loss: 0.57
Epoch: 10/50, Test acc: 65.13, Train acc: 67.21
Iteration: 0, Loss: 0.57
Epoch: 11/50, Test acc: 65.57, Train acc: 

In [79]:
testloader = DataLoader(test_ds, batch_size=32, shuffle=False)

In [80]:
evaluation(testloader,densenet201)

64.14473684210526

# Data1

In [81]:
data_dir='G:\DCDA-Net\Data_1'
print(os.listdir(data_dir))

['0', '1']


In [82]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [83]:
transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Resize(size=(224,224)),
                transforms.Normalize((0.3570, 0.7503, 0.5559), (0.3445, 0.0633, 0.2608))
                ])

In [84]:
dataset=ImageFolder(data_dir,transform=transform)
torch.manual_seed(42)
train_ds, val_ds, test_ds = random_split(dataset, [0.8,0.15,0.05])
len(train_ds),len(val_ds),len(test_ds)

(4867, 912, 304)

In [85]:
trainloader =DataLoader(train_ds, batch_size=16, shuffle=True,num_workers=4)
valloader = DataLoader(val_ds, batch_size=32, shuffle=True,num_workers=4)

In [86]:
def evaluation(dataloader, model):
    total, correct = 0, 0
    model.eval()
    for data in dataloader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, pred = torch.max(outputs.data, 1)

        total += labels.size(0)
        correct += (pred == labels).sum().item()
    return 100 * correct / total

In [87]:
def train_func(model, loss_fn, opt, trainloader, valloader, scheduler , epochs=50):
    loss_epoch_arr = []
    train_accuracy_lis = []
    val_accuracy_lis = []
    val_loss_epoch_arr = []

    max_epochs = epochs
    min_loss = 1000
    model.train()

    for epoch in notebook.tqdm(range(max_epochs), total=max_epochs, unit="epochs"):
        model.train()
        for i, data in enumerate(trainloader, 0):
    
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            opt.zero_grad()
            outputs = model(inputs)
            loss = loss_fn(outputs, labels)
            loss.backward()
            opt.step()
            scheduler.step()

            if min_loss > loss.item():
                min_loss = loss.item()
                best_model = copy.deepcopy(model.state_dict())
                print("Min loss %0.2f" %min_loss)
            del inputs, labels, outputs
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
            if i % 2000 == 0:
                print('Iteration: %d, Loss: %0.2f' % (i, loss.item()))
            
       
        loss_epoch_arr.append(loss.item())
        train_accuracy = evaluation(trainloader, model)
        val_accuracy = evaluation(valloader, model)
        print('Epoch: %d/%d, Test acc: %0.2f, Train acc: %0.2f' % (
              epoch, max_epochs, val_accuracy, train_accuracy))
        train_accuracy_lis.append(train_accuracy)
        val_accuracy_lis.append(val_accuracy)
    
        model.eval()
        with torch.no_grad():
            for i, data in enumerate(valloader, 0):
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = loss_fn(outputs, labels)

        del inputs, labels, outputs
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

        val_loss_epoch_arr.append(loss.item())
    return best_model, loss_epoch_arr, train_accuracy_lis, val_accuracy_lis, val_loss_epoch_arr

# Resnet50

In [88]:
resnet = models.resnet50(weights=True)

In [89]:
for param in resnet.parameters():
    param.requires_grad=False

In [90]:
num_features = resnet.fc.in_features
new_classifier = nn.Sequential(
    nn.Linear(num_features, 256), nn.ReLU(), nn.Dropout(0.2),
    nn.Linear(256, 2))
resnet.fc = new_classifier

In [91]:
resnet =resnet.to(device)
loss_fn = nn.CrossEntropyLoss()
opt = optim.Adam(resnet.parameters(), lr=0.01)
summary(resnet, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

In [92]:
gamma = 0.8  # The decay factor for each epoch
scheduler = lr_scheduler.ExponentialLR(opt, gamma=gamma)


In [93]:
best_model, loss_epoch_arr, train_accuracy_lis, val_accuracy_lis, val_loss_epoch_arr = train_func(resnet, loss_fn, opt, trainloader, valloader, scheduler , epochs=50
                                                                            )      

  0%|          | 0/50 [00:00<?, ?epochs/s]

Min loss 0.69
Iteration: 0, Loss: 0.69
Min loss 0.44
Min loss 0.42
Epoch: 0/50, Test acc: 49.78, Train acc: 51.12
Iteration: 0, Loss: 0.81
Epoch: 1/50, Test acc: 50.00, Train acc: 51.26
Iteration: 0, Loss: 1.19
Epoch: 2/50, Test acc: 49.56, Train acc: 51.00
Iteration: 0, Loss: 1.02
Min loss 0.34
Epoch: 3/50, Test acc: 49.67, Train acc: 50.96
Iteration: 0, Loss: 2.08
Epoch: 4/50, Test acc: 49.56, Train acc: 51.30
Iteration: 0, Loss: 1.20
Min loss 0.33
Epoch: 5/50, Test acc: 49.45, Train acc: 51.04
Iteration: 0, Loss: 1.34
Epoch: 6/50, Test acc: 49.56, Train acc: 50.75
Iteration: 0, Loss: 1.39
Epoch: 7/50, Test acc: 49.12, Train acc: 51.08
Iteration: 0, Loss: 0.70
Epoch: 8/50, Test acc: 49.23, Train acc: 51.02
Iteration: 0, Loss: 0.73
Epoch: 9/50, Test acc: 49.12, Train acc: 50.91
Iteration: 0, Loss: 1.10
Min loss 0.31
Min loss 0.17
Epoch: 10/50, Test acc: 48.68, Train acc: 50.69
Iteration: 0, Loss: 1.07
Epoch: 11/50, Test acc: 49.45, Train acc: 50.69
Iteration: 0, Loss: 1.11
Epoch: 12/5

In [94]:
testloader = DataLoader(test_ds, batch_size=32, shuffle=False)

In [95]:
evaluation(testloader,resnet)

49.3421052631579

# Resnet-152

In [96]:
resnet = models.resnet152(weights=True)

In [97]:
for param in resnet.parameters():
    param.requires_grad=False

In [98]:
num_features = resnet.fc.in_features
new_classifier = nn.Sequential(
    nn.Linear(num_features, 256), nn.ReLU(), nn.Dropout(0.2),
    nn.Linear(256, 2))
resnet.fc = new_classifier

In [99]:
resnet =resnet.to(device)
loss_fn = nn.CrossEntropyLoss()
opt = optim.Adam(resnet.parameters(), lr=0.01)
summary(resnet, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

In [100]:
gamma = 0.8  # The decay factor for each epoch
scheduler = lr_scheduler.ExponentialLR(opt, gamma=gamma)

In [101]:
best_model, loss_epoch_arr, train_accuracy_lis, val_accuracy_lis, val_loss_epoch_arr = train_func(resnet, loss_fn, opt, trainloader, valloader, scheduler , epochs=50
                                                                            )      

  0%|          | 0/50 [00:00<?, ?epochs/s]

Min loss 0.71
Iteration: 0, Loss: 0.71
Min loss 0.55
Min loss 0.48
Min loss 0.45
Epoch: 0/50, Test acc: 56.14, Train acc: 54.04
Iteration: 0, Loss: 0.73
Epoch: 1/50, Test acc: 55.15, Train acc: 54.12
Iteration: 0, Loss: 0.97
Epoch: 2/50, Test acc: 54.61, Train acc: 53.83
Iteration: 0, Loss: 0.62
Min loss 0.35
Epoch: 3/50, Test acc: 54.71, Train acc: 54.26
Iteration: 0, Loss: 0.82
Min loss 0.29
Epoch: 4/50, Test acc: 55.04, Train acc: 54.14
Iteration: 0, Loss: 1.36
Min loss 0.24
Epoch: 5/50, Test acc: 54.82, Train acc: 54.00
Iteration: 0, Loss: 1.14
Epoch: 6/50, Test acc: 54.39, Train acc: 54.22
Iteration: 0, Loss: 0.61
Epoch: 7/50, Test acc: 54.82, Train acc: 54.06
Iteration: 0, Loss: 0.45
Epoch: 8/50, Test acc: 55.04, Train acc: 54.06
Iteration: 0, Loss: 1.00
Epoch: 9/50, Test acc: 54.93, Train acc: 53.69
Iteration: 0, Loss: 1.19
Epoch: 10/50, Test acc: 55.48, Train acc: 54.24
Iteration: 0, Loss: 1.08
Epoch: 11/50, Test acc: 56.14, Train acc: 54.10
Iteration: 0, Loss: 1.38
Epoch: 12/5

In [102]:
testloader = DataLoader(test_ds, batch_size=32, shuffle=False)

In [103]:
evaluation(testloader,resnet)

55.26315789473684

# Resnet 18

In [104]:
resnet = models.resnet18(weights=True)

In [105]:
for param in resnet.parameters():
    param.requires_grad=False

In [106]:
num_features = resnet.fc.in_features
new_classifier = nn.Sequential(
    nn.Linear(num_features, 256), nn.ReLU(), nn.Dropout(0.2),
    nn.Linear(256, 2))
resnet.fc = new_classifier

In [107]:
resnet =resnet.to(device)
loss_fn = nn.CrossEntropyLoss()
opt = optim.Adam(resnet.parameters(), lr=0.01)
summary(resnet, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
       BasicBlock-11           [-1, 64, 56, 56]               0
           Conv2d-12           [-1, 64, 56, 56]          36,864
      BatchNorm2d-13           [-1, 64, 56, 56]             128
             ReLU-14           [-1, 64,

In [108]:
gamma = 0.8  # The decay factor for each epoch
scheduler = lr_scheduler.ExponentialLR(opt, gamma=gamma)

In [109]:
best_model, loss_epoch_arr, train_accuracy_lis, val_accuracy_lis, val_loss_epoch_arr = train_func(resnet, loss_fn, opt, trainloader, valloader, scheduler , epochs=50
                                                                            )      

  0%|          | 0/50 [00:00<?, ?epochs/s]

Min loss 0.67
Iteration: 0, Loss: 0.67
Min loss 0.58
Min loss 0.56
Min loss 0.54
Min loss 0.50
Min loss 0.48
Min loss 0.41
Epoch: 0/50, Test acc: 56.14, Train acc: 56.32
Iteration: 0, Loss: 0.87
Epoch: 1/50, Test acc: 56.69, Train acc: 56.28
Iteration: 0, Loss: 0.86
Min loss 0.39
Min loss 0.38
Epoch: 2/50, Test acc: 56.80, Train acc: 54.76
Iteration: 0, Loss: 0.85
Epoch: 3/50, Test acc: 56.91, Train acc: 55.19
Iteration: 0, Loss: 0.99
Epoch: 4/50, Test acc: 56.69, Train acc: 54.94
Iteration: 0, Loss: 0.99
Epoch: 5/50, Test acc: 56.36, Train acc: 55.48
Iteration: 0, Loss: 0.68
Epoch: 6/50, Test acc: 56.14, Train acc: 56.01
Iteration: 0, Loss: 1.06
Epoch: 7/50, Test acc: 57.02, Train acc: 55.97
Iteration: 0, Loss: 0.71
Epoch: 8/50, Test acc: 56.58, Train acc: 55.52
Iteration: 0, Loss: 0.78
Epoch: 9/50, Test acc: 56.47, Train acc: 55.29
Iteration: 0, Loss: 0.67
Epoch: 10/50, Test acc: 56.69, Train acc: 55.87
Iteration: 0, Loss: 0.81
Epoch: 11/50, Test acc: 56.80, Train acc: 55.11
Iteratio

In [110]:
testloader = DataLoader(test_ds, batch_size=32, shuffle=False)

In [111]:
evaluation(testloader,resnet)

57.56578947368421

# resnet-34

In [112]:
resnet = models.resnet34(weights=True)

In [113]:
for param in resnet.parameters():
    param.requires_grad=False

In [114]:
num_features = resnet.fc.in_features
new_classifier = nn.Sequential(
    nn.Linear(num_features, 256), nn.ReLU(), nn.Dropout(0.2),
    nn.Linear(256, 2))
resnet.fc = new_classifier

In [115]:
resnet =resnet.to(device)
loss_fn = nn.CrossEntropyLoss()
opt = optim.Adam(resnet.parameters(), lr=0.01)
summary(resnet, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
       BasicBlock-11           [-1, 64, 56, 56]               0
           Conv2d-12           [-1, 64, 56, 56]          36,864
      BatchNorm2d-13           [-1, 64, 56, 56]             128
             ReLU-14           [-1, 64,

In [116]:
gamma = 0.8  # The decay factor for each epoch
scheduler = lr_scheduler.ExponentialLR(opt, gamma=gamma)


In [117]:
best_model, loss_epoch_arr, train_accuracy_lis, val_accuracy_lis, val_loss_epoch_arr = train_func(resnet, loss_fn, opt, trainloader, valloader, scheduler , epochs=50
                                                                            )      

  0%|          | 0/50 [00:00<?, ?epochs/s]

Min loss 0.65
Iteration: 0, Loss: 0.65
Min loss 0.62
Min loss 0.58
Min loss 0.53
Min loss 0.45
Min loss 0.42
Epoch: 0/50, Test acc: 55.48, Train acc: 56.34
Iteration: 0, Loss: 1.18
Epoch: 1/50, Test acc: 54.61, Train acc: 56.15
Iteration: 0, Loss: 0.72
Epoch: 2/50, Test acc: 54.61, Train acc: 55.29
Iteration: 0, Loss: 0.78
Min loss 0.35
Epoch: 3/50, Test acc: 54.61, Train acc: 55.74
Iteration: 0, Loss: 0.89
Epoch: 4/50, Test acc: 54.61, Train acc: 55.64
Iteration: 0, Loss: 0.74
Epoch: 5/50, Test acc: 55.26, Train acc: 55.68
Iteration: 0, Loss: 0.75
Epoch: 6/50, Test acc: 55.04, Train acc: 55.33
Iteration: 0, Loss: 1.12
Epoch: 7/50, Test acc: 54.17, Train acc: 56.05
Iteration: 0, Loss: 0.81
Epoch: 8/50, Test acc: 55.70, Train acc: 56.26
Iteration: 0, Loss: 0.86
Epoch: 9/50, Test acc: 54.71, Train acc: 55.50
Iteration: 0, Loss: 0.84
Epoch: 10/50, Test acc: 54.17, Train acc: 55.15
Iteration: 0, Loss: 0.87
Epoch: 11/50, Test acc: 55.15, Train acc: 55.93
Iteration: 0, Loss: 0.64
Epoch: 12/5

In [118]:
testloader = DataLoader(test_ds, batch_size=32, shuffle=False)

In [119]:
evaluation(testloader,resnet)

55.26315789473684

# resnet 101

In [120]:
resnet = models.resnet101(weights=True)

In [121]:
for param in resnet.parameters():
    param.requires_grad=False

In [122]:
num_features = resnet.fc.in_features
new_classifier = nn.Sequential(
    nn.Linear(num_features, 256), nn.ReLU(), nn.Dropout(0.2),
    nn.Linear(256, 2))
resnet.fc = new_classifier

In [123]:
resnet =resnet.to(device)
loss_fn = nn.CrossEntropyLoss()
opt = optim.Adam(resnet.parameters(), lr=0.01)
summary(resnet, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

In [124]:
gamma = 0.8  # The decay factor for each epoch
scheduler = lr_scheduler.ExponentialLR(opt, gamma=gamma)


In [125]:
best_model, loss_epoch_arr, train_accuracy_lis, val_accuracy_lis, val_loss_epoch_arr = train_func(resnet, loss_fn, opt, trainloader, valloader, scheduler , epochs=50
                                                                            )      

  0%|          | 0/50 [00:00<?, ?epochs/s]

Min loss 0.72
Iteration: 0, Loss: 0.72
Min loss 0.52
Min loss 0.45
Min loss 0.43
Min loss 0.40
Epoch: 0/50, Test acc: 53.73, Train acc: 54.63
Iteration: 0, Loss: 1.70
Epoch: 1/50, Test acc: 55.04, Train acc: 55.95
Iteration: 0, Loss: 0.47
Min loss 0.35
Epoch: 2/50, Test acc: 53.84, Train acc: 55.48
Iteration: 0, Loss: 1.34
Epoch: 3/50, Test acc: 54.61, Train acc: 55.11
Iteration: 0, Loss: 2.11
Min loss 0.19
Epoch: 4/50, Test acc: 53.62, Train acc: 53.89
Iteration: 0, Loss: 1.84
Epoch: 5/50, Test acc: 54.61, Train acc: 55.64
Iteration: 0, Loss: 0.78
Epoch: 6/50, Test acc: 55.04, Train acc: 54.94
Iteration: 0, Loss: 0.70
Epoch: 7/50, Test acc: 54.06, Train acc: 55.17
Iteration: 0, Loss: 1.42
Epoch: 8/50, Test acc: 54.61, Train acc: 54.90
Iteration: 0, Loss: 1.10
Epoch: 9/50, Test acc: 53.84, Train acc: 54.72
Iteration: 0, Loss: 1.09
Epoch: 10/50, Test acc: 54.82, Train acc: 54.30
Iteration: 0, Loss: 1.25
Epoch: 11/50, Test acc: 55.59, Train acc: 55.50
Iteration: 0, Loss: 1.21
Epoch: 12/5

In [126]:
testloader = DataLoader(test_ds, batch_size=32, shuffle=False)

In [127]:
evaluation(testloader,resnet)

57.89473684210526

# VGG-16

In [128]:
vgg16 = models.vgg16_bn(pretrained=True)

In [129]:
for param in vgg16.features.parameters():
    param.requires_grad = False

In [130]:
num_features = vgg16.classifier[6].in_features
vgg16.classifier[6] = nn.Linear(num_features, 2)

In [131]:
vgg16 = vgg16.to(device)
loss_fn = nn.CrossEntropyLoss()
opt = optim.Adam(vgg16.parameters(), lr=0.01)

In [132]:
gamma = 0.8  # The decay factor for each epoch
scheduler = lr_scheduler.ExponentialLR(opt, gamma=gamma)

In [133]:
best_model, loss_epoch_arr, train_accuracy_lis, val_accuracy_lis, val_loss_epoch_arr = train_func(vgg16, loss_fn, opt, trainloader, valloader, scheduler , epochs=50
                                                                            )      

  0%|          | 0/50 [00:00<?, ?epochs/s]

Min loss 0.82
Iteration: 0, Loss: 0.82
Epoch: 0/50, Test acc: 48.79, Train acc: 50.54
Iteration: 0, Loss: 31.94
Epoch: 1/50, Test acc: 48.90, Train acc: 50.65
Iteration: 0, Loss: 15.54
Epoch: 2/50, Test acc: 48.90, Train acc: 50.69
Iteration: 0, Loss: 13.60
Min loss 0.40
Epoch: 3/50, Test acc: 48.90, Train acc: 50.63
Iteration: 0, Loss: 27.05
Min loss 0.00
Epoch: 4/50, Test acc: 48.90, Train acc: 50.61
Iteration: 0, Loss: 35.51
Epoch: 5/50, Test acc: 48.90, Train acc: 50.57
Iteration: 0, Loss: 27.22
Epoch: 6/50, Test acc: 48.79, Train acc: 50.54
Iteration: 0, Loss: 16.12
Epoch: 7/50, Test acc: 48.90, Train acc: 50.61
Iteration: 0, Loss: 12.94
Epoch: 8/50, Test acc: 48.79, Train acc: 50.57
Iteration: 0, Loss: 21.96
Epoch: 9/50, Test acc: 49.01, Train acc: 50.71
Iteration: 0, Loss: 37.53
Epoch: 10/50, Test acc: 49.01, Train acc: 50.65
Iteration: 0, Loss: 12.29
Epoch: 11/50, Test acc: 48.90, Train acc: 50.61
Iteration: 0, Loss: 32.53
Epoch: 12/50, Test acc: 48.90, Train acc: 50.63
Iterati

In [134]:
testloader = DataLoader(test_ds, batch_size=32, shuffle=False)

In [135]:
evaluation(testloader,vgg16)

45.06578947368421

# VGG-19

In [136]:
vgg19 = models.vgg19_bn(pretrained=True)

In [137]:
for param in vgg19.features.parameters():
    param.requires_grad = False

In [138]:
num_features = vgg19.classifier[6].in_features
vgg19.classifier[6] = nn.Linear(num_features, 2)

In [139]:
vgg19 = vgg19.to(device)
loss_fn = nn.CrossEntropyLoss()
opt = optim.Adam(vgg19.parameters(), lr=0.01)

In [140]:
gamma = 0.8  # The decay factor for each epoch
scheduler = lr_scheduler.ExponentialLR(opt, gamma=gamma)

In [141]:
best_model, loss_epoch_arr, train_accuracy_lis, val_accuracy_lis, val_loss_epoch_arr = train_func(vgg19, loss_fn, opt, trainloader, valloader, scheduler , epochs=50
                                                                            )      

  0%|          | 0/50 [00:00<?, ?epochs/s]

Min loss 0.76
Iteration: 0, Loss: 0.76
Epoch: 0/50, Test acc: 48.25, Train acc: 49.72
Iteration: 0, Loss: 13.54
Epoch: 1/50, Test acc: 48.25, Train acc: 49.76
Iteration: 0, Loss: 12.70
Epoch: 2/50, Test acc: 48.25, Train acc: 49.76
Iteration: 0, Loss: 21.00
Epoch: 3/50, Test acc: 48.25, Train acc: 49.76
Iteration: 0, Loss: 14.70
Epoch: 4/50, Test acc: 48.25, Train acc: 49.78
Iteration: 0, Loss: 31.21
Epoch: 5/50, Test acc: 48.25, Train acc: 49.78
Iteration: 0, Loss: 49.23
Epoch: 6/50, Test acc: 48.25, Train acc: 49.78
Iteration: 0, Loss: 23.92
Epoch: 7/50, Test acc: 48.25, Train acc: 49.78
Iteration: 0, Loss: 4.83
Epoch: 8/50, Test acc: 48.25, Train acc: 49.78
Iteration: 0, Loss: 26.74
Epoch: 9/50, Test acc: 48.25, Train acc: 49.76
Iteration: 0, Loss: 29.02
Epoch: 10/50, Test acc: 48.25, Train acc: 49.76
Iteration: 0, Loss: 24.68
Min loss 0.00
Epoch: 11/50, Test acc: 48.25, Train acc: 49.80
Iteration: 0, Loss: 26.46
Epoch: 12/50, Test acc: 48.25, Train acc: 49.76
Iteration: 0, Loss: 28

In [142]:
testloader = DataLoader(test_ds, batch_size=32, shuffle=False)

In [143]:
evaluation(testloader,vgg19)

45.06578947368421

# Densenet121

In [144]:
densenet121 = models.densenet121(pretrained=True)

In [145]:
for param in densenet121.features.parameters():
    param.requires_grad = False


In [146]:
num_features = densenet121.classifier.in_features
densenet121.classifier = nn.Linear(num_features, 2)

In [147]:
densenet121 = densenet121.to(device)
loss_fn = nn.CrossEntropyLoss()
opt = optim.Adam(densenet121.parameters(), lr=0.01)

In [148]:
gamma = 0.8  # The decay factor for each epoch
scheduler = lr_scheduler.ExponentialLR(opt, gamma=gamma)

In [149]:
best_model, loss_epoch_arr, train_accuracy_lis, val_accuracy_lis, val_loss_epoch_arr = train_func(densenet121, loss_fn, opt, trainloader, valloader, scheduler , epochs=50
                                                                            )      

  0%|          | 0/50 [00:00<?, ?epochs/s]

Min loss 0.69
Iteration: 0, Loss: 0.69
Min loss 0.61
Min loss 0.59
Min loss 0.55
Min loss 0.53
Min loss 0.52
Min loss 0.49
Epoch: 0/50, Test acc: 55.04, Train acc: 52.37
Iteration: 0, Loss: 0.80
Epoch: 1/50, Test acc: 54.28, Train acc: 52.04
Iteration: 0, Loss: 0.72
Epoch: 2/50, Test acc: 55.04, Train acc: 52.27
Iteration: 0, Loss: 0.71
Min loss 0.46
Epoch: 3/50, Test acc: 55.04, Train acc: 52.39
Iteration: 0, Loss: 0.78
Epoch: 4/50, Test acc: 54.06, Train acc: 52.21
Iteration: 0, Loss: 0.66
Epoch: 5/50, Test acc: 54.50, Train acc: 52.33
Iteration: 0, Loss: 0.77
Epoch: 6/50, Test acc: 54.93, Train acc: 52.37
Iteration: 0, Loss: 0.66
Epoch: 7/50, Test acc: 53.95, Train acc: 52.41
Iteration: 0, Loss: 0.60
Epoch: 8/50, Test acc: 54.50, Train acc: 51.86
Iteration: 0, Loss: 0.76
Epoch: 9/50, Test acc: 53.84, Train acc: 52.46
Iteration: 0, Loss: 0.86
Epoch: 10/50, Test acc: 54.39, Train acc: 52.04
Iteration: 0, Loss: 0.68
Epoch: 11/50, Test acc: 53.84, Train acc: 52.00
Iteration: 0, Loss: 0.

In [150]:
testloader = DataLoader(test_ds, batch_size=32, shuffle=False)

In [151]:
evaluation(testloader,densenet121)

55.5921052631579

# DenseNet201

In [152]:
densenet201 = models.densenet201(pretrained=True)

In [153]:
for param in densenet121.features.parameters():
    param.requires_grad = False

In [154]:
num_features = densenet201.classifier.in_features
densenet201.classifier = nn.Linear(num_features, 2)

In [155]:
densenet201 = densenet201.to(device)
loss_fn = nn.CrossEntropyLoss()
opt = optim.Adam(densenet201.parameters(), lr=0.01)

In [156]:
gamma = 0.8  # The decay factor for each epoch
scheduler = lr_scheduler.ExponentialLR(opt, gamma=gamma)

In [157]:
best_model, loss_epoch_arr, train_accuracy_lis, val_accuracy_lis, val_loss_epoch_arr = train_func(densenet201, loss_fn, opt, trainloader, valloader, scheduler , epochs=50
                                                                            )      

  0%|          | 0/50 [00:00<?, ?epochs/s]

Min loss 0.76
Iteration: 0, Loss: 0.76
Min loss 0.73
Min loss 0.57
Epoch: 0/50, Test acc: 50.11, Train acc: 49.83
Iteration: 0, Loss: 0.75
Epoch: 1/50, Test acc: 49.78, Train acc: 49.91
Iteration: 0, Loss: 0.68
Epoch: 2/50, Test acc: 50.22, Train acc: 49.64
Iteration: 0, Loss: 0.69
Epoch: 3/50, Test acc: 48.36, Train acc: 49.58
Iteration: 0, Loss: 0.60
Epoch: 4/50, Test acc: 50.00, Train acc: 49.76
Iteration: 0, Loss: 0.67
Epoch: 5/50, Test acc: 48.46, Train acc: 49.62
Iteration: 0, Loss: 0.72
Epoch: 6/50, Test acc: 49.34, Train acc: 49.78
Iteration: 0, Loss: 0.69
Epoch: 7/50, Test acc: 49.34, Train acc: 49.58
Iteration: 0, Loss: 0.69
Epoch: 8/50, Test acc: 48.79, Train acc: 49.60
Iteration: 0, Loss: 0.77
Epoch: 9/50, Test acc: 49.56, Train acc: 49.58
Iteration: 0, Loss: 0.72
Epoch: 10/50, Test acc: 49.89, Train acc: 49.64
Iteration: 0, Loss: 0.78
Epoch: 11/50, Test acc: 50.00, Train acc: 49.76
Iteration: 0, Loss: 0.68
Min loss 0.51
Epoch: 12/50, Test acc: 48.57, Train acc: 49.43
Itera

In [158]:
testloader = DataLoader(test_ds, batch_size=32, shuffle=False)

In [159]:
evaluation(testloader,densenet201)

46.05263157894737